In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt


In [2]:
import os
import shutil

# Directorio de entrada
input_dir = 'imagenes_filtradas'

# Directorios de salida
output_dir = 'user_database'
os.makedirs(output_dir, exist_ok=True)

# Organizar las imágenes en carpetas por usuario
for image_name in os.listdir(input_dir):
    if image_name.endswith('.png'):
        user_id = image_name[:2]  # Extraer el identificador del usuario (ejemplo: "11", "12")
        user_dir = os.path.join(output_dir, f"user{user_id}")
        os.makedirs(user_dir, exist_ok=True)
        shutil.copy(os.path.join(input_dir, image_name), os.path.join(user_dir, image_name))

print("Base de datos creada con éxito.")


Base de datos creada con éxito.


In [3]:
def extract_features(image_path):
    """
    Extrae puntos clave y descriptores de una imagen usando SIFT.
    """
    # Leer la imagen en escala de grises
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Error al cargar la imagen: {image_path}")
    
    # Crear el objeto SIFT
    sift = cv2.SIFT_create()
    
    # Detectar puntos clave y calcular descriptores
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors


In [4]:
def match_fingerprints(descriptors1, descriptors2, ratio_threshold=0.75):
    """
    Compara dos conjuntos de descriptores SIFT y devuelve la cantidad de coincidencias.
    """
    # Crear un objeto de coincidencia BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    
    # Encontrar las coincidencias iniciales
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    # Filtrar coincidencias usando el ratio de Lowe
    good_matches = [m for m, n in matches if m.distance < ratio_threshold * n.distance]
    
    return len(good_matches)


In [5]:
def visualize_matches(image1_path, image2_path, ratio_threshold=0.75):
    """
    Muestra las coincidencias entre dos imágenes de huellas.
    """
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    sift = cv2.SIFT_create()

    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)
    good_matches = [m for m, n in matches if m.distance < ratio_threshold * n.distance]

    match_img = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(match_img, cmap='gray')
    plt.title(f"Coincidencias: {len(good_matches)}")
    plt.show()


In [6]:
def authenticate_user(new_fingerprint_path, database_dir, match_threshold=20):
    """
    Compara una nueva huella con la base de datos organizada por usuario.
    """
    # Extraer características de la nueva huella
    _, new_descriptors = extract_features(new_fingerprint_path)
    if new_descriptors is None:
        return "No se encontraron características en la huella nueva."
    
    max_matches = 0
    best_user = None

    # Recorrer cada usuario en la base de datos
    for user_dir in os.listdir(database_dir):
        user_path = os.path.join(database_dir, user_dir)
        if os.path.isdir(user_path):
            for fingerprint_path in os.listdir(user_path):
                full_path = os.path.join(user_path, fingerprint_path)
                _, db_descriptors = extract_features(full_path)

                if db_descriptors is None:
                    continue
                
                # Comparar descriptores
                match_count = match_fingerprints(new_descriptors, db_descriptors)
                print(f"Comparando con {full_path}, Coincidencias: {match_count}")

                if match_count > max_matches:
                    max_matches = match_count
                    best_user = user_dir

    if max_matches >= match_threshold:
        return f"Huella autenticada como del usuario {best_user} con {max_matches} coincidencias."
    else:
        return f"Huella rechazada. Máximas coincidencias: {max_matches}"


In [7]:
# Crear la base de datos organizada por usuario
database_dir = 'user_database'

# Ruta de una nueva huella
new_fingerprint_path = 'imagenes_filtradas/20f_01.png'  # Cambia al nombre de la huella que quieres autenticar

# Autenticar la huella
result = authenticate_user(new_fingerprint_path, database_dir)
print(result)


Comparando con user_database\user11\11f_01.png, Coincidencias: 60
Comparando con user_database\user11\11s_01.png, Coincidencias: 47
Comparando con user_database\user12\12f_01.png, Coincidencias: 54
Comparando con user_database\user12\12s_01.png, Coincidencias: 61
Comparando con user_database\user13\13f_01.png, Coincidencias: 46
Comparando con user_database\user13\13s_01.png, Coincidencias: 44
Comparando con user_database\user14\14f_01.png, Coincidencias: 48
Comparando con user_database\user14\14s_01.png, Coincidencias: 48
Comparando con user_database\user15\15f_01.png, Coincidencias: 57
Comparando con user_database\user15\15s_01.png, Coincidencias: 54
Comparando con user_database\user16\16f_01.png, Coincidencias: 64
Comparando con user_database\user16\16s_01.png, Coincidencias: 56
Comparando con user_database\user17\17f_01.png, Coincidencias: 55
Comparando con user_database\user17\17s_01.png, Coincidencias: 60
Comparando con user_database\user18\18f_01.png, Coincidencias: 51
Comparando